In [51]:
import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()

print(bucket)

sagemaker-us-east-1-011113936377


In [52]:
!aws s3 sync s3://{bucket}/wsb/data/ data/

In [53]:
import sys
import re
import os
import json
import datetime
import pytz
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

In [54]:
nltk.download('punkt')
nltk.download('wordnet')
token_pattern = re.compile(r"(?u)\b\w\w+\b")

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc.lower()) if len(t) >= 2 and re.match("[a-z].*",t) 
                and re.match(token_pattern, t) and t.upper() not in allsymbols]


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
ET = pytz.timezone('US/Eastern')

ignorelist=["DD","FREE","CASH","ON","I"]
allsymbols=[]
with open("allsymbols.txt") as fh:
  allsymbols=fh.readlines()
allsymbols = [x.strip() for x in allsymbols]

dt=datetime.timedelta(days=7)

wnl = WordNetLemmatizer()


In [56]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [57]:
import datetime

stockdata={}
def getstockprice(symbol,date):
    if symbol not in stockdata:
        with open("stockprices/"+symbol,"r") as fh:
            data=json.load(fh)
        stockdata[symbol]={}
        for stockday in data:
            indate=datetime.datetime.strptime(stockday["begins_at"],"%Y-%m-%dT%H:%M:%SZ").date()
            stockdata[symbol][indate]=stockday
    today=datetime.date.today()
    while True:
        #print(date)
        if date in stockdata[symbol]:
            return float(stockdata[symbol][date]['close_price'])
        date=date+datetime.timedelta(days=1)
        if date>today:
            break
    raise Exception("No stock data found")
    return None
            

In [58]:
files = os.listdir("data/")

foundsymbols=[]
relevant_posts = []
i=0
for file in files:
  if(round(i/100,0)==i/100):
        update_progress(i / len(files))  
  with open("data/"+file,"r") as fh:
    try:
        data=json.load(fh)
        text = (data["title"]+" "+data["selftext"]).replace('\"','').replace('\'','')
        allmatches=[]
        matches=re.findall('\W*([A-Z][A-Z\.]{0,3})\W',text)
        for submatch in matches:
            allmatches.append({submatch:submatch})
            #pass
        matches2=re.findall('\W*(\$[a-z\.]{1,4})',text)
        for submatch in matches2:
            allmatches.append({submatch.upper()[1:]:submatch})
        thesesymbols=[]
        #print(allmatches)
        for submatch in allmatches:
            symbol=list(submatch.keys())[0]
            #print(submatch.keys())
            #print(symbol)
            #print([list(x.keys())[0] for x in thesesymbols])
            if symbol in allsymbols and symbol not in [list(x.keys())[0] for x in thesesymbols] and symbol not in ignorelist:
                thesesymbols.append(submatch)
                if symbol not in foundsymbols:
                    foundsymbols.append(symbol)
        if len(thesesymbols)>0:
            relevant_posts.append({"data":data,"symbols":thesesymbols})
            #print(thesesymbols)
    except Exception as err:
      print("Error with",file)
      raise(err)
  i+=1
    

  #if i>1000:
  #  break



Progress: [####################] 99.9%


In [59]:
print(len(foundsymbols))
with open("foundsymbols.txt","w") as fh:
  fh.write("\n".join(foundsymbols))
foundsymbols

3661


['TSLA',
 'B',
 'AMZN',
 'COST',
 'C',
 'LULU',
 'AMD',
 'FB',
 'PTON',
 'AAPL',
 'GOLD',
 'ROKU',
 'NFLX',
 'DIS',
 'HAS',
 'GOOD',
 'ONCY',
 'SPCE',
 'AMC',
 'KR',
 'PM',
 'ABC',
 'X',
 'Y',
 'CYBR',
 'PANW',
 'RPD',
 'EDIT',
 'DTE',
 'DCOM',
 'AT',
 'ACB',
 'THC',
 'MU',
 'IP',
 'AG',
 'GILD',
 'TJX',
 'HTZ',
 'TVTY',
 'DGX',
 'QQQ',
 'TA',
 'BYND',
 'POST',
 'SR',
 'NIO',
 'RING',
 'A',
 'NEW',
 'IT',
 'FOSL',
 'BK',
 'PINS',
 'KRTX',
 'OVID',
 'DOW',
 'SBUX',
 'VC',
 'MCD',
 'MSFT',
 'WMT',
 'SNAP',
 'PECK',
 'HUGE',
 'GDP',
 'WELL',
 'CDC',
 'BA',
 'V',
 'FAT',
 'NOC',
 'LMT',
 'RH',
 'AAL',
 'GO',
 'SDC',
 'ALGN',
 'ONE',
 'ATVI',
 'M',
 'CEO',
 'K',
 'ALLY',
 'SIX',
 'CME',
 'AM',
 'LIVE',
 'FIT',
 'SWBI',
 'EC',
 'F',
 'CRM',
 'BAC',
 'CTO',
 'PEP',
 'UAL',
 'DAL',
 'JNJ',
 'PCG',
 'E',
 'PLUS',
 'AMTD',
 'TTD',
 'DEA',
 'CBD',
 'CCL',
 'ARE',
 'OUT',
 'G',
 'BMO',
 'IONS',
 'FOR',
 'HAL',
 'INFY',
 'KO',
 'SYF',
 'NDAQ',
 'NEE',
 'ERIC',
 'DPZ',
 'LUV',
 'UNP',
 'AXP',
 'SNY'

In [60]:
#split into multiple corpus here


import random
random.shuffle(relevant_posts)
n_train = int(0.8 * len(relevant_posts))

training_posts = relevant_posts[:n_train]
test_posts = relevant_posts[n_train:]
val_posts = test_posts[:n_train//2]
test_posts = test_posts[n_train//2:]

vocab_input = [t["data"]["title"]+" "+t["data"]["selftext"] for t in training_posts]
print(len(relevant_posts),n_train)

34504 27603


In [61]:
test_posts = relevant_posts[n_train:]
print(n_train,n_train//2)
test_posts = relevant_posts[n_train:]
print(len(test_posts))
print(len(test_posts)//2)
val_posts = test_posts[:len(test_posts)//2]
test_posts = test_posts[len(test_posts)//2:]
len(test_posts)

27603 13801
6901
3450


3451

In [62]:
vocab_input[0]

"Proof you don’t need to know what you’re doing   \n\n![img](ftbthfhyniy31)\n\nI found this sub about 18 months ago and created my RH account 13 months ago. \n\nOptions\n\nI have no idea what I’m doing and because of that I typically look for options in the $100-$1,000 range. I play options off the industries I know and if I get a good or bad feeling. Then I look at the historical averages to see what’s feasible and I play my options further out (3-6-9 months). I go by the wsb theory of, “if it’s good enough to screenshot, it’s good enough to sell.” If I see an option go up 40%+, I sell, or I’ll sell enough to cover the initial purchase and let some ride (this is honestly the hardest part but just hit the sell button…if you start thinking, “well, what if…” then you’re in trouble). Most of the options I’ve sold for a profit go up even more, but I try not to look. For example, I bought 3 $160 calls (exp 3/20) on Disney for $248 a few weeks ago, today I sold one option at 300% and another

In [63]:
vocab_size=5000

vectorizer = CountVectorizer(input='content', analyzer='word', stop_words='english', tokenizer=LemmaTokenizer(), max_features=vocab_size, max_df=0.95, min_df=2)

vectors = vectorizer.fit_transform(vocab_input)
vocab_list = vectorizer.get_feature_names()
print('vocab size:', len(vocab_list))
idx = np.arange(vectors.shape[0])
np.random.shuffle(idx)
vectors = vectors[idx]


vocab size: 5000


In [64]:
vocab_list

['abbott',
 'ability',
 'able',
 'aboard',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'accelerate',
 'accelerated',
 'accelerating',
 'acceleration',
 'accept',
 'accepted',
 'accepting',
 'access',
 'accessory',
 'accident',
 'accidentally',
 'according',
 'accordingly',
 'account',
 'accounted',
 'accounting',
 'accurate',
 'accurately',
 'achieve',
 'achieved',
 'ackman',
 'acquire',
 'acquired',
 'acquires',
 'acquiring',
 'acquisition',
 'act',
 'acted',
 'acting',
 'action',
 'active',
 'actively',
 'activision',
 'activist',
 'activity',
 'actual',
 'actually',
 'ad',
 'adam',
 'adapt',
 'add',
 'added',
 'addict',
 'addicted',
 'addiction',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'adj',
 'adjust',
 'adjusted',
 'adjustment',
 'administration',
 'admit',
 'adoption',
 'adult',
 'advance',
 'advanced',
 'advantage',
 'adverse',
 'advertisement',
 'advertiser',
 'advertising',
 'advice',
 'advise',
 'advised',
 'advisor',
 'advisory',
 'aerospac

In [65]:
threshhold=.05
lt = LemmaTokenizer()
from scipy.sparse import csr_matrix

def calculateData(post):
    #print(post)
    data=post["data"]
    thesesymbols=post["symbols"]
    text = data["title"]+" "+data["selftext"]
    texttokens=lt(text)
    #print(text,texttokens)
    #print("Symbol:",thesesymbols)
    wordvector=[]
    for word in vocab_list:
        wordvector.append(texttokens.count(word))
    returnarray=[]
    for symbolmap in thesesymbols:
        symbol=list(symbolmap.keys())[0]
        try:
        #if True:
            
                #if word in texttokens:
                    #print(word)
                #    wordvector.append(1)
                #else:
                #    wordvector.append(0)
                #locword = text.find(word)
                #distance=0
                #if locword>-1:
                #    distance=abs(text.find(symbolmap[symbol])-locword)
                #    print(symbol,word,distance)
                #wordvector.append(distance)
            #print(symbol,wordvector)
            posttime=[0,0,0,0]
            start=ET.localize(datetime.datetime.fromtimestamp(data["created_utc"]))
            if start.hour>=21:
                posttime[0]=1
            elif start.hour < 5:
                posttime[0]=1
            elif start.hour < 9:
                posttime[1]=1
            elif start.hour < 16:
                posttime[2]=1
            else:
                posttime[3]=1
            if start.hour>=9:
              start=start+datetime.timedelta(days=1)
            start=start.replace(hour=9,minute=0,second=0,microsecond=0).date()
            end=start+dt
            startprice=getstockprice(symbol,start)
            endprice=getstockprice(symbol,end)
            #print(start,end,history)
            delta=(endprice-startprice)/startprice
            if delta<=-1*(threshhold):
                result=1
            elif delta>=threshhold:
                result=2
                #print(symbol,delta,result,wordvector)
            else:
                result=0
            returnarray.append([result]+posttime+wordvector)
        except Exception as err:
            print("Error with",symbol,", skipping this one:",err)
    return returnarray



training_vector = []
i=0
for posts in training_posts:
    #try:
    if True:
       response=calculateData(posts)
       for row in response:
            training_vector.append(row)
    #except Exception as err:
    #    print("Error",err)
    #    pass
    i+=1
    if(round(i/100,0)==i/100):
        update_progress(i / len(training_posts))
        print(len(training_posts))



Progress: [####################] 100.0%
27603


In [66]:
import pandas as pd
pd.DataFrame(training_vector, columns=["Output"]+["21-4","5-8","9-15","16-20"]+vocab_list)

,Output,21-4,5-8,9-15,16-20,abbott,ability,able,aboard,absolute,...,yr/yr,ytd,yuan,zealand,zero,zillow,zombie,zone,zoom,zynga
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68581,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68582,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68583,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68584,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
import boto3
import sagemaker.amazon.common as smac
import io

np_training_vector = np.array(training_vector).astype('float32')

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, np_training_vector[:,1:], np_training_vector[:,0])
buf.seek(0)

prefix = 'wsb-training'
key= 'wsb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))


output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))





uploaded training data location: s3://sagemaker-us-east-1-011113936377/wsb-training/train/wsb-data
training artifacts will be uploaded to: s3://sagemaker-us-east-1-011113936377/wsb-training/output


In [ ]:


from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

from sagemaker import get_execution_role

role = get_execution_role()
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.m4.10xlarge',
                                       output_path=output_location,
                                       sagemaker_session=session)
linear.set_hyperparameters(feature_dim=len(vocab_list)+4,
                           predictor_type='multiclass_classifier',
                           num_classes=3,
                           mini_batch_size=200)

linear.fit({'train': s3_train_data})

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-11 11:24:11 Starting - Starting the training job...
2020-08-11 11:24:13 Starting - Launching requested ML instances......
2020-08-11 11:25:27 Starting - Preparing the instances for training.........
2020-08-11 11:26:56 Downloading - Downloading input data...
2020-08-11 11:27:32 Training - Downloading the training image..
2020-08-11 11:27:54 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[08/11/2020 11:27:59 INFO 140556035622720] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5'

In [1]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.10xlarge')

NameError: name 'linear' is not defined

In [ ]:
test_vector = []
i=0
for posts in test_posts:
    #try:
    if True:
       response=calculateData(posts)
       for row in response:
            test_vector.append(row)
    #except Exception as err:
    #    print("Error",err)
    #    pass
    i+=1
    if True:
        update_progress(i / len(test_posts))
        print(len(test_posts))
np_test_vector = np.array(test_vector).astype('float32')

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [ ]:
np_test_vector[:4,1:]

In [ ]:
result = linear_predictor.predict(np_training_vector[:4,1:])
print(result)

In [ ]:
import numpy as np

predictions = []
for array in np.array_split(np_test_vector[:,1:], 100):
    #print(array)
    result = linear_predictor.predict(array)
    #print(result)
    predictions += [r['predicted_label'] for r in result['predictions']]
#print(predictions)
predictions = np.array(predictions)


In [ ]:
test_labels=[]
for value in np_test_vector[:,0]:
    #print(value[0])
    test_labels.append(value)
print(len(test_labels),len(predictions))

In [ ]:
import pandas as pd

pd.crosstab([test_labels], predictions, rownames=['actuals'], colnames=['predictions'])

In [ ]:
import numpy
def error_rate(predictions, labels):
    """Return the error rate and confusions."""
    correct = numpy.sum(predictions == labels)
    total = predictions.shape[0]

    error = 100.0 - (100 * float(correct) / float(total))

    confusions = numpy.zeros([3,3], numpy.int32)
    bundled = zip(predictions, labels)
    for predicted, actual in bundled:
        confusions[int(predicted), int(actual)] += 1
    
    return error, confusions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

NUM_LABELS = 3  # change it according to num_class in your dataset
test_error, confusions = error_rate(numpy.asarray(predictions), numpy.asarray(test_labels))
print('Test error: %.1f%%' % test_error)

plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid(False)
plt.xticks(numpy.arange(NUM_LABELS))
plt.yticks(numpy.arange(NUM_LABELS))
plt.imshow(confusions, cmap=plt.cm.jet, interpolation='nearest');

for i, cas in enumerate(confusions):
    for j, count in enumerate(cas):
        if count > 0:
            xoff = .07 * len(str(count))
            plt.text(j-xoff, i+.2, int(count), fontsize=9, color='white')

In [ ]:

sklearn.metrics.f1_score(numpy.asarray(test_labels),numpy.asarray(predictions),average=None)

In [ ]:
sklearn.metrics.recall_score(numpy.asarray(test_labels),numpy.asarray(predictions),average=None)

In [ ]:
sklearn.metrics.precision_score(numpy.asarray(test_labels),numpy.asarray(predictions),average=None)

In [ ]:
print(sklearn.metrics.classification_report(numpy.asarray(test_labels),numpy.asarray(predictions)))

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(linear_predictor.endpoint)